In [1]:
import pandas as pd
import numpy as np
import os
import tensorflow as tf
import time
from sklearn.model_selection import train_test_split
from tensorflow.keras import Model
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout
import matplotlib.pyplot as plt
from sklearn.metrics import confusion_matrix
from tensorflow.keras.models import load_model, save_model

# Set random seeds for reproducibility
np.random.seed(42)
tf.random.set_seed(42)

print("✅ 1. Necessary libraries imported and random seeds set.")


✅ 1. Necessary libraries imported and random seeds set.


In [2]:
# --- Data and Paths (Relative) ---
# Load input data
DATA_CSV_PATH = 'Input_data_CNN.csv' 
# Model save path (relative)
CKPT_PATH = "CNN_model_output"

# --- Data Processing ---
# Labels must match the 'state' column in your CSV
LABELS = ['enter', 'leave', 'stay', 'out'] 
# The size of the sliding window
WINDOW_SIZE = 6 
# The columns to be used as features
FEATURE_COLUMNS = ['RSSI1', 'RSSI2', 'RSSI3', 'RSSI4', 'RSSI5',
                 'RSSI6', 'RSSI7', 'RSSI8', 'RSSI9', 'RSSI10']
TARGET_COLUMN = 'state'
# UPDATE: Removed 'DAY_COLUMN' as it's no longer in the CSV

# --- Model Hyperparameters ---
NN_CONN2D_1 = 64      # Neurons for Conv2D Layer 1
NN_CONN2D_2 = 128     # Neurons for Conv2D Layer 2
NN_NH_1 = 256         # Neurons for Fully Connected Layer
LEARNING_RATE = 0.0001
BATCH_SIZE = 100
SAVE_STEP_EPOCHS = 3000 # Total epochs to train
DISPLAY_STEP = 100    # How often to print loss/accuracy

# Create the output directory if it doesn't exist
if not os.path.exists(CKPT_PATH):
    os.makedirs(CKPT_PATH)
    print(f"Created model output directory at: {CKPT_PATH}")
else:
    print(f"Model output directory already exists at: {CKPT_PATH}")

print("\n✅ 2. Configuration and hyperparameters loaded.")
print(f"   - Data file: {DATA_CSV_PATH}")
print(f"   - Model path: {CKPT_PATH}")
print(f"   - Labels: {LABELS}")
print(f"   - Epochs: {SAVE_STEP_EPOCHS}, Batch Size: {BATCH_SIZE}")

Model output directory already exists at: CNN_model_output

✅ 2. Configuration and hyperparameters loaded.
   - Data file: Input_data_CNN.csv
   - Model path: CNN_model_output
   - Labels: ['enter', 'leave', 'stay', 'out']
   - Epochs: 3000, Batch Size: 100


In [3]:
def load_and_process_data(csv_path, feature_cols, target_col, labels_list, window_size):
    """
    Loads data from a single CSV and creates sliding windows across the entire file.
    Assumes the data is a single, continuous time series.
    """
    try:
        df = pd.read_csv(csv_path)
    except FileNotFoundError:
        print(f"Error: Data file not found at {csv_path}")
        print("Please make sure 'Input_data_CNN.csv' is in the same directory.")
        return None, None
        
    print(f"Loaded data with {len(df)} rows.")
    
    all_X_data = []
    all_y_data = []
    
    print(f"Applying sliding window (size={window_size}) across all data...")
    
    # We need enough rows to make at least one window
    if len(df) < window_size:
        print("Error: Data is shorter than window size. Cannot create samples.")
        return None, None
            
    # Apply the sliding window logic across the entire dataframe
    for i in range(len(df) - window_size + 1):
        # The label is taken from the *last* row of the window
        label_value = df.iloc[i + window_size - 1][target_col]
        
        # Skip if label is NaN or not in our defined LABELS list
        if pd.isna(label_value) or label_value not in labels_list:
            continue
            
        # Get the window of feature data (6 rows, 10 columns)
        window_features = df.iloc[i : i + window_size][feature_cols].to_numpy()
        
        # --- Preprocessing ---
        # 1. Normalize RSSI values from [-100, 0] to [0, 1]
        normalized_window = (window_features + 100.0) / 100.0
        
        # 2. Reshape for CNN: (height, width, channels) -> (6, 10, 1)
        reshaped_window = normalized_window.reshape(window_size, len(feature_cols), 1)
        
        # --- One-hot encode label ---
        label = np.zeros(len(labels_list))
        label_index = labels_list.index(label_value)
        label[label_index] = 1
        
        all_X_data.append(reshaped_window)
        all_y_data.append(label)

    if not all_X_data:
        print("Error: No valid windows were created. Please check your data and configuration.")
        return None, None
        
    # Convert lists to final NumPy arrays
    X = np.array(all_X_data)
    Y = np.array(all_y_data)
    
    return X, Y

print("✅ 3. Data loading and preprocessing function defined (load_and_process_data).")
print("   - NOTE: This version assumes a single continuous data file (no 'day' grouping).")

✅ 3. Data loading and preprocessing function defined (load_and_process_data).
   - NOTE: This version assumes a single continuous data file (no 'day' grouping).


In [4]:
# 1. Load and process the data
X_data, Y_data = load_and_process_data(
    csv_path=DATA_CSV_PATH,
    feature_cols=FEATURE_COLUMNS,
    target_col=TARGET_COLUMN,
    labels_list=LABELS,
    window_size=WINDOW_SIZE
)

# 2. Split the processed data into training and testing sets
if X_data is not None:
    X_train, X_test, Y_train, Y_test = train_test_split(
        X_data, Y_data, test_size=0.2, random_state=42, stratify=Y_data
    )
    
    # Print shapes to verify
    print(f"\nTotal samples: {len(X_data)}")
    print(f"X_train shape: {X_train.shape}")
    print(f"Y_train shape: {Y_train.shape}")
    print(f"X_test shape:  {X_test.shape}")
    print(f"Y_test shape:  {Y_test.shape}")
    
    # Calculate total batches
    total_batch = int(len(X_train) / BATCH_SIZE)
    print(f"Total training batches per epoch: {total_batch}")
    
    print("\n✅ 4. Data loaded, processed, and split into training/testing sets.")
else:
    print("\n❌ 4. Data loading failed. Please check errors above.")

Loaded data with 12093 rows.
Applying sliding window (size=6) across all data...

Total samples: 12088
X_train shape: (9670, 6, 10, 1)
Y_train shape: (9670, 4)
X_test shape:  (2418, 6, 10, 1)
Y_test shape:  (2418, 4)
Total training batches per epoch: 96

✅ 4. Data loaded, processed, and split into training/testing sets.


In [5]:
class MyModel(Model):
    def __init__(self):
        super(MyModel, self).__init__()
        # First convolutional layer (64 filters, 3x3 kernel)
        self.conv1 = Conv2D(NN_CONN2D_1, 3, activation='relu', padding='same')
        self.pool1 = MaxPooling2D(2)  # Pooling layer
        
        # Second convolutional layer (128 filters, 2x2 kernel)
        self.conv2 = Conv2D(NN_CONN2D_2, 2, activation='relu')
        self.pool2 = MaxPooling2D(1)  # Pooling layer
        
        self.flatten = Flatten()  # Flatten the 3D output to 1D
        self.d1 = Dense(NN_NH_1, activation='relu')  # Fully connected layer
        self.dropout = Dropout(0.5)  # Dropout for regularization
        self.d2 = Dense(len(LABELS))  # Output layer (logits)

    def call(self, x, training=False):
        x = self.conv1(x)
        x = self.pool1(x)
        x = self.conv2(x)
        x = self.pool2(x)
        x = self.flatten(x)
        x = self.d1(x)
        if training:
            # Apply dropout only during training
            x = self.dropout(x, training=training)
        return self.d2(x)

# Create an instance of the model
model = MyModel()

# Build the model to see the summary (optional but recommended in notebooks)
# Input shape is (Batch Size, Height, Width, Channels)
# We check if X_train exists before trying to build
if 'X_train' in locals() and len(X_train) > 0:
    model.build(input_shape=(None, WINDOW_SIZE, len(FEATURE_COLUMNS), 1))
    model.summary()
    print("\n✅ 5. CNN model (MyModel) defined, built, and summary printed.")
else:
    print("❌ 5. Model defined, but cannot build or show summary because data (X_train) is not loaded.")




Model: "my_model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d (Conv2D)             multiple                  640       
                                                                 
 max_pooling2d (MaxPooling2  multiple                  0         
 D)                                                              
                                                                 
 conv2d_1 (Conv2D)           multiple                  32896     
                                                                 
 max_pooling2d_1 (MaxPoolin  multiple                  0         
 g2D)                                                            
                                                                 
 flatten (Flatten)           multiple                  0         
                                                                 
 dense (Dense)               multiple                  

In [6]:
# Define the loss function
# from_logits=True because our model outputs raw scores (logits), not probabilities
loss_object = tf.keras.losses.CategoricalCrossentropy(from_logits=True)

# Define the optimizer
optimizer = tf.keras.optimizers.Adam(LEARNING_RATE)

# Define metrics to measure loss and accuracy
train_loss = tf.keras.metrics.Mean(name='train_loss')
train_accuracy = tf.keras.metrics.CategoricalAccuracy(name='train_accuracy')

# Use @tf.function to compile the training step into a high-performance graph
@tf.function
def train_step(images, labels):
    with tf.GradientTape() as tape:
        # Forward pass
        predictions = model(images, training=True)
        # Calculate loss
        loss = loss_object(labels, predictions)
    
    # Calculate gradients
    gradients = tape.gradient(loss, model.trainable_variables)
    # Apply gradients to update weights
    optimizer.apply_gradients(zip(gradients, model.trainable_variables))
    
    # Update metrics
    train_loss(loss)
    train_accuracy(labels, predictions)

print("✅ 6. Loss function, optimizer, metrics, and @tf.function train_step defined.")

✅ 6. Loss function, optimizer, metrics, and @tf.function train_step defined.


In [7]:
# Only run training if data was loaded correctly
if 'X_train' in locals() and len(X_train) > 0:
    print(f"Starting training for {SAVE_STEP_EPOCHS} epochs...")
    startTime = time.time()

    # Start training loop
    for epoch in range(SAVE_STEP_EPOCHS):
        
        # Reset metrics at the start of each epoch
        train_loss.reset_states()
        train_accuracy.reset_states()
        
        # Shuffle the training data
        indices = np.arange(len(X_train))
        np.random.shuffle(indices)
        X_train_shuffled = X_train[indices]
        Y_train_shuffled = Y_train[indices]

        for batch in range(total_batch):
            # Get one batch of data
            start_idx = batch * BATCH_SIZE
            end_idx = (batch + 1) * BATCH_SIZE
            batch_xs, batch_ys = X_train_shuffled[start_idx:end_idx], Y_train_shuffled[start_idx:end_idx]
            
            # Run the training step
            if len(batch_xs) > 0: # Avoid running on empty batch
                train_step(batch_xs, batch_ys)

        # Print progress
        if (epoch + 1) % DISPLAY_STEP == 0 or epoch == SAVE_STEP_EPOCHS - 1:
            template = 'Epoch {}, Loss: {:.4f}, Accuracy: {:.2f}%'
            print(template.format(epoch + 1,
                                 train_loss.result(),
                                 train_accuracy.result() * 100))

    # Save the final model
    model.save(os.path.join(CKPT_PATH, "model"), save_format="tf")

    # Training finished
    print(f"\nTrain finished! Total training time: {time.time() - startTime:.2f} seconds")
    print(f"Final model saved to: {os.path.join(CKPT_PATH, 'model')}")
    print("\n✅ 7. Model training complete and final model saved.")

else:
    print("❌ 7. Training skipped because training data is not available.")

Starting training for 3000 epochs...
Epoch 100, Loss: 0.4761, Accuracy: 83.10%
Epoch 200, Loss: 0.3641, Accuracy: 87.42%
Epoch 300, Loss: 0.2888, Accuracy: 90.10%
Epoch 400, Loss: 0.2458, Accuracy: 91.79%
Epoch 500, Loss: 0.2238, Accuracy: 92.33%
Epoch 600, Loss: 0.2097, Accuracy: 92.93%
Epoch 700, Loss: 0.2034, Accuracy: 93.17%
Epoch 800, Loss: 0.1971, Accuracy: 93.38%
Epoch 900, Loss: 0.1956, Accuracy: 93.40%
Epoch 1000, Loss: 0.1901, Accuracy: 93.60%
Epoch 1100, Loss: 0.1901, Accuracy: 93.53%
Epoch 1200, Loss: 0.1881, Accuracy: 93.67%
Epoch 1300, Loss: 0.1882, Accuracy: 93.61%
Epoch 1400, Loss: 0.1862, Accuracy: 93.64%
Epoch 1500, Loss: 0.1856, Accuracy: 93.66%
Epoch 1600, Loss: 0.1846, Accuracy: 93.75%
Epoch 1700, Loss: 0.1861, Accuracy: 93.65%
Epoch 1800, Loss: 0.1843, Accuracy: 93.69%
Epoch 1900, Loss: 0.1838, Accuracy: 93.73%
Epoch 2000, Loss: 0.1848, Accuracy: 93.68%
Epoch 2100, Loss: 0.1832, Accuracy: 93.72%
Epoch 2200, Loss: 0.1851, Accuracy: 93.68%
Epoch 2300, Loss: 0.1832, 

INFO:tensorflow:Assets written to: CNN_model_output\model\assets



Train finished! Total training time: 2100.22 seconds
Final model saved to: CNN_model_output\model

✅ 7. Model training complete and final model saved.


In [8]:
# Only run evaluation if data was loaded correctly
if 'X_test' in locals() and len(X_test) > 0:
    print("Running evaluation on the test set...")
    
    # Use the trained model to make predictions on the test set
    predictions_logits = model.predict(X_test)

    # To get the final class prediction, find the index of the highest score (argmax)
    predicted_classes = np.argmax(predictions_logits, axis=1)
    # Get the true classes from the one-hot encoded Y_test
    true_classes = np.argmax(Y_test, axis=1)

    # Calculate accuracy
    accuracy = np.mean(predicted_classes == true_classes)
    print(f"   - Test Accuracy: {accuracy:.4f}")

    # Calculate the confusion matrix
    cm = confusion_matrix(true_classes, predicted_classes)
    print("   - Confusion matrix calculated.")
    print("\n✅ 8. Model evaluation complete.")
else:
    print("❌ 8. Evaluation skipped because test data is not available.")

Running evaluation on the test set...
76/76 [==============================] - 0s 3ms/step
   - Test Accuracy: 0.8544
   - Confusion matrix calculated.

✅ 8. Model evaluation complete.


In [9]:
# Only run evaluation if data was loaded correctly
if 'X_test' in locals() and len(X_test) > 0:
    print("Running evaluation on the test set...")
    
    # Use the trained model to make predictions on the test set
    predictions_logits = model.predict(X_test)

    # To get the final class prediction, find the index of the highest score (argmax)
    predicted_classes = np.argmax(predictions_logits, axis=1)
    # Get the true classes from the one-hot encoded Y_test
    true_classes = np.argmax(Y_test, axis=1)

    # Calculate accuracy
    accuracy = np.mean(predicted_classes == true_classes)
    print(f"   - Test Accuracy: {accuracy:.4f}")

    # Calculate the confusion matrix
    cm = confusion_matrix(true_classes, predicted_classes)
    print("   - Confusion matrix calculated.")
    print("\n✅ 8. Model evaluation complete.")
else:
    print("❌ 8. Evaluation skipped because test data is not available.")

Running evaluation on the test set...
76/76 [==============================] - 0s 3ms/step
   - Test Accuracy: 0.8544
   - Confusion matrix calculated.

✅ 8. Model evaluation complete.
